In [1]:
import torch
from models import Discriminator, ContentEncoder, StyleEncoder, Decoder
from losses import DisLoss

In [2]:
discriminator = Discriminator()
content_encoder = ContentEncoder(G_ch=64)
style_encoder = StyleEncoder(G_ch=64)
decoder = Decoder(G_ch=64, nEmbedding=1024)

Param count for Ds initialized parameters: 19541696
Param count for Ds initialized parameters: 20591296
Param count for Ds initialized parameters: 27289027


In [3]:
cont, residulte_features = content_encoder(torch.randn(1, 3, 128, 128))
style_emb, style_fc, residual_features_style = style_encoder(torch.randn(1, 3, 128, 128))
img_print2write = decoder(cont, residulte_features, style_emb, style_fc, residual_features_style)

In [4]:
out = discriminator(img_print2write)

In [6]:
criterionD = DisLoss()

In [7]:
criterionD(out, True)

tensor(1.1544, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [8]:
criterionD(out, False)

tensor(0.3787, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)